<a href="https://colab.research.google.com/github/sungjin417/sesac-phython/blob/master/%EA%B5%90)%EB%8C%80%EA%B7%9C%EB%AA%A8_LLM%EC%9D%84_%ED%99%9C%EC%9A%A9%ED%95%9C_%EC%A7%80%EC%8B%9D_%EC%B1%97%EB%B4%87_%EA%B0%9C%EB%B0%9C_3%EC%9D%BC%EC%B0%A8(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# 🌼 대규모 LLM을 활용한 지식 챗봇 개발(1) - 3차시(24.11.25)

---


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import warnings
warnings.filterwarnings(action='ignore')

In [6]:
import numpy as np
import tensorflow as tf
import transformers
import pandas as pd

In [7]:
comment=pd.read_excel('/content/drive/MyDrive/A_comment_train.xlsx',index_col=0) # index_col=0 : 0번째를 인덱스로 삼겠다

In [8]:
comment.head()

,data,label
0,재미는 있는데 시간이 짧은게 아쉽네요~,긍정
1,"OO 관련 내용은 우리 직원과는 거리가 멀었음, 특히, 사내에 홍보할 내용은 아니라고 봄",부정
2,스토리가 너무 딱딱해서 별로였음,부정
3,프로그램A 화이팅하세요!!,긍정
4,높은 곳에 올라가는 모습이 너무 위험해 보여요.,부정


In [9]:
comment.isnull().sum() # NaN 값 없음

,0
data,0
label,0


In [10]:
comment['label'] = comment['label'].replace(['부정', '긍정'], [0,1])

In [11]:
comment.tail()

,data,label
96,작년에 프로그램A를 재밋게 봤던 시청자로서 올해의 미니드라마도 매우 기대가 됩니다....,1
97,프로그램C 잘 보았습니다. 모든일의 바탕은 안전인것 같습니다. 모두를 보호하는 최고...,1
98,위험한 시설에 대한 설명도 부탁드립니다,0
99,구체적으로 어떤 활동을 해왔었고 앞으로 어떤활동을 할건지 잘 설명해줬으면 좋았을 것...,0
100,우리 회사가 잘되면 나라도 잘된다. 회사 화이팅 나라 화이팅,1


In [12]:
comment.info()

<class 'pandas.core.frame.DataFrame'>
Index: 352 entries, 0 to 100
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   data    352 non-null    object
 1   label   352 non-null    int64 
dtypes: int64(1), object(1)
memory usage: 8.2+ KB


In [13]:
X = comment.data.to_list()
y = comment.label.to_list()

In [14]:
from sklearn.model_selection import train_test_split

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y, test_size = 0.2, random_state = 42) # stratify = y : y를 기준으로 데이터셋을 균등하게 나누기

In [16]:
len(X_train), len(y_train)

(281, 281)

In [17]:
len(X_test), len(y_test)

(71, 71)

In [18]:
X_train[:2]

['서비스가 차별화되고 좋은 것 같습니다.', '잘들었습니다. 다음 화도 기대하겠습니다. ']

In [19]:
# Pre-Trained된 BERT tekenizer 가져오기
from transformers import BertTokenizerFast
bert_model = 'klue/bert-base'
tokenizer = BertTokenizerFast.from_pretrained(bert_model)

tokenizer_config.json:   0%|          | 0.00/289 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/248k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/495k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/425 [00:00<?, ?B/s]

In [20]:
tokenizer.vocab_size # 기존에 학습했던 모델의 vocab_size

32000

In [21]:
tokenizer.vocab # #은 앞에 다른게 나온다는 의미

{'유산소': 24019,
 '조각상': 27016,
 '등지': 9134,
 '대유': 28984,
 '뱀': 1135,
 '둘러앉': 27058,
 '올바': 7030,
 '##입': 2372,
 '캄캄': 18813,
 '##임자': 14641,
 '뒤처지': 28373,
 '머뭇거리': 20411,
 '촉발': 13564,
 '윤성': 18774,
 '##시계': 16355,
 '떠오를': 22582,
 '무공': 13211,
 '아카': 7899,
 '문': 1091,
 '센티미터': 19562,
 '돌려야': 29935,
 '##냉장고': 27775,
 '[unused278]': 31778,
 '고민': 4445,
 '요구': 3917,
 '##옷': 2428,
 '담요': 20931,
 '타임스': 9387,
 'Power': 28739,
 '나간': 7623,
 '호두': 15043,
 '저버리': 23400,
 '##히드': 27186,
 '##사람': 11336,
 '언어': 4686,
 '뒤돌': 17327,
 '고객': 3916,
 '##섹': 2594,
 '리원': 26620,
 '카멜': 30322,
 '세션': 15678,
 '##⠀⠀⠀⠀⠀⠀⠀⠀': 15996,
 '번번': 16918,
 '발자': 10199,
 '서늘': 16796,
 '적군': 19831,
 '서기': 8907,
 '담판': 27181,
 '영업자': 30793,
 '말뚝': 26216,
 '주택가': 18341,
 '가단': 29616,
 '동경': 11136,
 '면회': 17212,
 '난쟁이': 26770,
 '돌릴': 16856,
 '목화': 26912,
 '끊임없': 12652,
 '밋밋': 23977,
 '팽': 1843,
 '나중': 5294,
 '매회': 28164,
 '##애나': 20005,
 '##60': 9616,
 '미상': 30819,
 '##주의자': 9183,
 '양식': 6277,
 '기차역': 30909,
 '까페': 31101,

In [22]:
X_train_enc = tokenizer(X_train, truncation=True, padding=True)
# truncation : vocab_size를 초과하는 길이를 갖는 문장은 자동으로 잘라주기
# padding : 가장 긴 문장에 맞춰 알아서 패딩해 주기

In [23]:
X_test_enc = tokenizer(X_test, truncation=True, padding=True)

In [24]:
X_train_enc.keys()

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])

In [25]:
print(X_train_enc['input_ids'][0])
# 정수 인덱싱 + 토큰화 + 패딩

[2, 3838, 2116, 4874, 2267, 2496, 2088, 1560, 2073, 575, 555, 2219, 3606, 18, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [26]:
print(X_train_enc['token_type_ids'][0]) # 한문장 이기 때문에 0으로 출력

[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [27]:
# token_type_ids : bert모델에서 사용하는 feature (2개의 문장을 넣었을때 0과 1로 구분)
test = tokenizer('오늘 날씨가 좋다', '내일도 좋을까?', return_tensors = 'pt')
# [CLS] 오늘 날씨가 좋다 [SEP] 내일도 좋을까?
# 토큰은 다른 값이 들어갈 수 있음
print(test['token_type_ids'])

tensor([[0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1]])


In [28]:
train_dataset = tf.data.Dataset.from_tensor_slices((dict(X_train_enc), y_train)) # dict(X_trina_enc) : 에러 방지로 원래 딕셔너리 형태지만, 다시한번 딕셔너리라고 명시

In [29]:
train_dataset = train_dataset.shuffle(1000).batch(16).cache().prefetch(tf.data.experimental.AUTOTUNE)

In [30]:
test_dataset = tf.data.Dataset.from_tensor_slices((dict(X_test_enc), y_test))
test_dataset = test_dataset.batch(16).cache().prefetch(tf.data.experimental.AUTOTUNE) # test_dataset에 셔플을 하지 않은 이유 결과에 영향을 주지 않기 때문에

In [31]:
from transformers import TFBertForSequenceClassification
model = TFBertForSequenceClassification.from_pretrained(bert_model, num_labels=2, from_pt=True) # num_labels=2 : 출력 클래스 갯수를 2개로

pytorch_model.bin:   0%|          | 0.00/445M [00:00<?, ?B/s]

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [32]:
optimizer = tf.keras.optimizers.Adam(learning_rate = 5e-5)

In [33]:
model.compile(optimizer = optimizer, loss = model.hf_compute_loss, metrics=['accuracy'])

In [34]:
model.fit(train_dataset, epochs = 3, batch_size = 16, validation_data = (test_dataset))

Epoch 1/3
18/18 [==============================] - 413s 21s/step - loss: 0.3008 - accuracy: 0.8719 - val_loss: 0.1462 - val_accuracy: 0.9577
Epoch 2/3
18/18 [==============================] - 371s 21s/step - loss: 0.0134 - accuracy: 1.0000 - val_loss: 0.2031 - val_accuracy: 0.9437
Epoch 3/3
18/18 [==============================] - 369s 20s/step - loss: 0.0388 - accuracy: 0.9929 - val_loss: 0.0892 - val_accuracy: 0.9859


In [35]:
y_test_pred = model.predict(test_dataset)

5/5 [==============================] - 22s 3s/step


In [36]:
y_test_pred.logits.shape # logits : 특정 클래스에 속할 확률 (batch 크기, 클래스 수)

(71, 2)

In [37]:
y_test_pred.logits[:10]

array([[ 2.862325 , -2.3111925],
       [-3.4855332,  2.721464 ],
       [-2.8120122,  2.1571307],
       [ 3.587555 , -2.7049627],
       [-2.0853746,  1.5323702],
       [-3.6793432,  2.753895 ],
       [-3.5400264,  2.8695374],
       [ 3.1818938, -2.7786922],
       [-2.8819366,  2.1138892],
       [-3.1003766,  2.273784 ]], dtype=float32)

In [39]:
df = pd.DataFrame(np.argmax(y_test_pred.logits, axis = 1), columns = ['predict'])  # 위의 수중에 왼쪽이 0, 오른쪽이 1 그중에서 큰 값을 출력
df

,predict
0,0
1,1
2,1
3,0
4,1
...,...
66,1
67,0
68,1
69,0


In [44]:
df ['true'] = y_test
df

,predict,ture,true
0,0,1,1
1,1,1,1
2,1,1,1
3,0,0,0
4,1,1,1
...,...,...,...
66,1,1,1
67,0,0,0
68,1,1,1
69,0,0,0


In [45]:
np.sum(df['true'] == df['predict'])/len(df)

0.9859154929577465